
## Agenten
KI Agenten sind laut vielen Experten das nächste große Ding in Sachen KIs. Ganz unabhängig davon, ob das stimmt, können (autonome) KI Agenten hilfreich sein und es lohnt sich, sich damit auseinander zu setzen.
Meiner Einschätzung nach erweitern KI Agenten das Spektrum an Werkzeugen, die man zum Lösen von Aufgaben benötigt. Diese ermöglichen auch Aufgaben zu löschen, die zuvor eigentlich nicht automatisierbar waren.

Dennoch, bei aller Euphorie, muss man sagen, dass Stand jetzt autonome Agenten noch in den Kinderschuhen stecken und häufig Fehler begehen. Komplexe Aufgaben sind also noch in weiter Ferne.

Heute schauen wir uns an, wie man Agenten erstellt und wozu sie gut sein könnten. Wir beginnen mit sehr einfachen Formen und steigern uns dann und setzen das Agentenframework LangGraph ein.


## Was sind denn Agenten überhaupt?

Die Definitionen, was ein Agent ist, gehen wie so oft stark auseinander. Wo sich die meisten jedoch einig sind, ist dass wenn man der KI Werkzeuge zur Verfügung stellt und die KI selbst entscheidet, wann sie diese Werkzeuge einsetzen möchte, die erste Stufe eines Agenten erreicht ist.
Es gibt aber je nach Definition auch Agenten ohne Werkzeuge, also z.B. Agenten, die durch Prompting-Techniken erzeugt werden. 

Meist weist man einem Agenten eine Rolle zu, z.B. Organisator, Qualitätssicherer, Coder usw. Der Hintergrund ist der, dass Sprachmodelle häufig besser performen, wenn man sie in eine Rolle versetzt. In Agentensystemen, hat man also meist ein paar Agenten mit unterschiedlichen Rollen.
In diesen Systemen tauschen die Agenten Daten aus, sei es dadurch, dass sie einen Prompt mit allen notwenigen Informationen bekommen und die Antwort wird z.B. einen anderen Agenten übergeben, oder z.B. durch einen gemeinsamen Speicher, auf den jeder Agent Zugriff hat.

Gibt man einer KI die Möglichkeit Werkzeuge zu nutzen, läuft das häufig so ab, dass die KI die Ausführung anfragt und jemand oder etwas führt das Werkzeug aus und übergibt das Ergebnis wiederum an die KI, die dann damit arbeiten kann. D.h. die KI kann nicht einfach selbst etwas ausführen. Allerings bieten Agentensysteme die Möglichkeit an, dass diese Tool-Anfragen automatisch ausgeführt werden, ohne dass ein Mensch die Ausführung genehmigen muss oder gar selbst ausführen muss. Hier sprechen wir dann von *autonomen* Agenten. Häufig kombiniert man das aber mit _Human-In-The-Loop_ Ansätzen. D.h. bei kritischen Ausführungen wird ein Mensch erstmal um Erlaubnis gebeten. Auch wird das genutzt um z.B. zu entscheiden, wie etwas weitergehen soll, im Sinne von entweder Weg A, B oder C z.B. Oder um weitere Informationen abzufragen.

Bei Systemen mit mehreren Agenten kann man für jeden Agent unterschiedliche Modelle verwenden. So kann man für jede Aufgabe das beste Modell auswählen. Während z.B. der Organisator-Agent ein Modell einsetzt, das gut logisch kombinieren kann, setzt der Coder ein Modell ein, das besonders gut Code produzieren kann.




## Tools

Wie funktioniert das mit den Tools? Die KI, also das Sprachmodell, kann nicht selbst Funktionen oder ähnliches ausführen. Aber die KI kann darum bitten.

Wenn man der KI also sagt, dass ihr Tools zur Verfügung stehen und man außerdem beschreibt, welche Argumente die Tools benötigen, dann kann die KI sich dafür entscheiden. In der Tat werden die Modelle darauf trainiert. Der Chat-Request an Chat-GPT (also die direkte OpenAI-Schnittstelle) enthält sogar einen besonderen Bereich außerhalb des Prompts für die Tools. Man übergibt dabei einen Namen des Tools und ganz wichtig eine Beschreibung, wofür das Tool gut sein soll. Die Beschreibung dient der KI dazu zu verstehen, was das Tool kann. Dann listet man die erwarteten Parameter auf. Auch hier mit Namen, Datentyp und Beschreibung. Und klar, die Beschreibung erklärt der KI wozu der Parameter gut ist.


Das Ganze können wir mal kurz über einen simplen Prompt simulieren. Nehmen wir das allererste Beispiel, das wir programmiert haben und passen es an.

In [ ]:
!pip install langchain langchain_openai python-dotenv

In [ ]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")

# model = ChatOpenAI(model="gpt-4o")
model = AzureChatOpenAI(openai_api_version="2024-05-01-preview", azure_deployment="gpt-4o", temperature=0.5)

messages = [
    SystemMessage("Du bist ein hilfreicher Chatbot. Falls du nach der Uhrzeit gefragt wirst, dann kannst du ein Tool namens 'datetime' verwenden."
                  "Wenn du das Tool brauchst, dann antworte ausschließlich mit 'datetime'."),
    HumanMessage("Wieviel Uhr ist es?"),
]

response_text = model.invoke(messages)

if response_text.content == "datetime":
    import datetime
    current_time = datetime.datetime.now().strftime("%d-%m-%Y %H:%M")

    messages.append(AIMessage(f"Antwort vom Tool ist: {current_time}. Beantworte jetzt die Frage des Nutzers"))

    response_text = model.invoke(messages)

response_text.content



Das funktioniert zwar, aber es wäre doch sehr mühsam, wenn wir das jedes mal so machen müssten.

Lasst uns also anschauen, was LangChain zu bieten hat. Hierfür werden wir allerdings noch nicht die Agenten-Funktionalität nutzen.
Das Stichwort heißt _Tools_. Wir können also Tools, also Werkzeuge, anlegen bzw. anbieten und LangChain kümmert sich um den Rest. Es gibt wie immer zahlreiche fertige Tools, die man einsetzen kann.

Erstmal müssen wir aber etwas installieren, nämlich DuckDuckGo-Search. Das wollen wir nämlich als Tool in unserer kleinen Anwendung nutzen!

In [ ]:
!pip install duckduckgo-search

Erstmal starten wir mit der Initialisierung. Wir importieren das Tool, das wir gleich nutzen möchten und legen uns davon eine Instanz an.
Wir testen auch, ob das Tool auch so funktioniert, d.h. ohne es in den Workflow mit der KI einzubinden.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.run("Der schnellste Vogel der Welt")

Das ist schon mal cool, aber etwas mehr Informationen wären auch gut.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.run("Der schnellste Vogel der Welt")

Jetzt bekommen wir auch Quellenangaben!

Wie weiß jetzt die KI, wofür das Tool denn gut ist? Wir haben ja gelernt, dass jedes Tool eine Beschreibung braucht, damit die KI weiß wie und wofür es zu nutzen ist.
Lass uns das mal ansehen:

In [ ]:
search.description

In [ ]:
search.args

Ok bis jetzt haben wir das Tool praktisch selbst ausgeführt. Jetzt wollen wir es der KI an die virtuelle Hand geben. Wir werden später bei den Agenten sehen, dass es etwas eleganter geht, als das was gleich folgt.

Jetzt ist es nämlich so, dass nicht jedes KI-Modell so gut in der Lage ist mit Tools umzugehen. OpenAIs ChatGPT ist z.B. extra dafür trainiert worden und man hat den Tools sogar einen extra Platz in der API gewidmet. Da kann man nämlich neben der Prompthistorie auch Tools mitgeben. Dort heißen sie allerdings _Functions_.
Die _Functions_ sind eine Liste von Funktionen, wie der Name schon andeutet, wo man Name, Beschreibung und Parameter angibt. Vermutlich werden diese Angaben irgendwie automatisch in den System-Prompt der KI eingefügt, sodass die KI weiß welche Tools da sind.

LangChain können wir dazu bringen, diese Funktionalität zu nutzen. Das schauen wir uns jetzt mal an...

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function


tools = [search]
functions = [convert_to_openai_function(t) for t in tools]
functions[0]

Ok, also offenbar gibt es eine Konvererfunktion, die aus den LangChain-Tools OpenAI-Functions macht.

Jetzt können wir die _Functions_ auch bei der Konversation übergeben. Jetzt brauchen wir nur einen Prompt, der die KI dazu veranlasst dieses Tool zu nutzen und nicht einfach aus dem eigenen Trainingsdaten antwortet. Das geht am einfachsten, wenn man nach aktuellen Ereignissen fragt.

In [ ]:
message = model.invoke(
    [HumanMessage(content="Wie viel kostet das Maß Bier auf dem Münchener Oktoberfest im Jahr 2024?")], functions=functions
)

message

Ahh ok! Was ist passiert? Nun, ChatGPT hat erkannt, dass wir suchen müssen und möchte gerne, dass wir das Tool _duckduckgo_results_json_ ausführen. Es gibt uns auch dafür die notwendigen Parameter mit. 

Jetzt müssten wir eigentlich loslegen und das ganze ausprogrammieren. D.h. wir müssten uns eine Routine bauen, die erkennt, welche Funktion ausgeführt werden soll. Dann müsste das Ergebnis als _ToolMessage_ an die KI übergeben werden, die dann daraus ihre finale Antwort baut.

Wer möchte, kann das gerne als Übung machen. Aber das Ganze müssen wir doch nicht selbst erfinden, denn LangChain hat das ja schon eingebaut! Nämlich als Agent!

Hier geht es zum nächsten Abschnitt: Erster Agent: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ki-zeitalter/ai-agent-workshop/blob/main/06_erster_agent.ipynb)